In [1]:
!pip install -U -q keras-nlp
!pip install --upgrade keras
!pip install kagglehub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 6.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-tools 1.66.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.5 which is incompatible.
tensorflow-metadata 1.16.0 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 4.25.5 which is incompatible.


In [2]:
import os
from google.colab import userdata
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')

In [3]:
import kagglehub
kagglehub.login()

Kaggle credentials set.
Kaggle credentials successfully validated.


In [4]:
import os
os.environ["KERAS_BACKEND"] = "jax"

In [5]:
%%capture
import kagglehub

path = kagglehub.model_download("google/gemma/transformers/2b")

In [6]:
print("Path to model files:", path)

Path to model files: /root/.cache/kagglehub/models/google/gemma/transformers/2b/2


In [7]:
!pip install accelerate
!pip install transformers

!pip install sentence_transformers langchain langchain-community

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3070, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2863, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3061, in _parsed_pkg_info
    return self._pkg_info
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2863, in __getattr__
    raise AttributeError(attr)
AttributeError: _pkg_info. Did you mean: 'egg_info'?

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py",

In [8]:
!pip install qdrant-client pypdf

  Using cached protobuf-5.28.2-cp38-abi3-manylinux2014_x86_64.whl.metadata (592 bytes)
Using cached protobuf-5.28.2-cp38-abi3-manylinux2014_x86_64.whl (316 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.5
    Uninstalling protobuf-4.25.5:
      Successfully uninstalled protobuf-4.25.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-ai-generativelanguage 0.6.6 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.28.2 which is incompatible.
google-cloud-datastore 2.19.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.28.2 which is incompatible.
google-cloud-firestore 2.16.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.2

In [9]:
import re
import numpy as np
import pandas as pd

import torch

import transformers
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          pipeline
                         )

from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.vectorstores import Qdrant
from langchain.llms import HuggingFacePipeline
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA

from sentence_transformers import SentenceTransformer
import bitsandbytes as bnb

In [10]:
import warnings
warnings.filterwarnings('ignore')

In [11]:
torch.device("cuda")

device(type='cuda')

In [12]:
device = torch.device("cuda")

In [13]:
name = '/root/.cache/kagglehub/models/google/gemma/transformers/2b/2'
length = 512

In [14]:
!pip install -U bitsandbytes

In [15]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    name,
    quantization_config=bnb_config,
    device_map=device)

tokenizer = AutoTokenizer.from_pretrained(
    name,
    max_seq_length = length,
    device_map=device)

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [16]:
def tokenize(query):
    return tokenizer(query, return_tensors="pt", truncation=True, padding=True).to(device)

def generate_text(**prompt):
    res = model.generate(
    **prompt,
    max_new_tokens=length,
    do_sample=True)
    res = tokenizer.decode(res[0], skip_special_tokens=True)
    return res

In [17]:
query = "What is supervised learning?"
query = tokenize(query)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [18]:
print(generate_text(**query))

What is supervised learning? Why do more people think there is a need for it?

<h2>What is Supervised Learning?</h2>

<strong>Supervised learning</strong> is learning algorithms that get the input and output pairs and learn the mapping from the input to the output. It is also commonly referred to as machine learning or <strong>ML</strong>.

<h2>What is the difference between supervised and unsupervised learning?</h2>

While both are learning algorithms but supervised learning specifically tries to create a model that can be expected to work for the entire input range. And it tries to use the training data to figure out what the relationship is between those data. For more on this check out Why is ML a branch of AI? It has more details about the difference between supervised and unsupervised learning.

<h2>Why Supervised Learning is popular?</h2>

Supervised learning is popular because it is more reliable at modeling real-world problem which is known as the concept-based model.

Some of

In [19]:
!pip install opendatasets

In [20]:
import opendatasets as od

od.download("https://www.kaggle.com/datasets/paultimothymooney/cvpr-2019-papers")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: syedosamaalishah092
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/paultimothymooney/cvpr-2019-papers


100%|██████████| 4.50G/4.50G [00:39<00:00, 124MB/s] 


In [21]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader('/content/cvpr-2019-papers/CVPR2019/papers/Amodio_TraVeLGAN_Image-To-Image_Translation_by_Transformation_Vector_Learning_CVPR_2019_paper.pdf')
docs = loader.load()

In [22]:
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}

embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 600, chunk_overlap = 0)
all_splits = text_splitter.split_documents(docs)

qdrant_collection = Qdrant.from_documents(
    all_splits,
    embeddings,
    location=":memory:",
    collection_name="CVPR 2019",
)

qdrant_retriever = qdrant_collection.as_retriever()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [23]:
pipeline = transformers.pipeline(
    "text-generation",
    model=name,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device="cuda",
    max_new_tokens=1024
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [24]:
gemma_llm = HuggingFacePipeline(
    pipeline=pipeline,
    model_kwargs={"temperature": 0.0},
)

qa = RetrievalQA.from_chain_type(
    llm=gemma_llm,
    chain_type="stuff",
    retriever=qdrant_retriever
)

In [25]:
query = str('what is a TraVeLGAN')
print(qa.invoke(query)['result'])

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

to see whether the TraVeLGAN’s mapping preserves the se-
mantics in the abacus domain in a systematic way, too.
In Figure 5, we started with a crossword and created a
regular three-by-three grid of black squares by editing an
image from Figure S2. Then, systematically, we move a
white square around the grid through each of the nine po-
sitions. In each case, the TraVeLGAN generates an aba-
cus with a bead moving around the grid appropriately. Re-
markably, it even colors the bead to ﬁt with the neighboring
beads, which differ throughout the grid. Given that none

erate its paired image.
As a consequence of these differences, the TraVeLGAN is
better able to handle mappings between complex, hetero-
geneous domains that require signiﬁcant and diverse shape
changing.
By avoiding direct regularization of the generators, the
TraVe

In [26]:
query = str('what happens in a TraVeLGAN')
print(qa.invoke(query)['result'])

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

to see whether the TraVeLGAN’s mapping preserves the se-
mantics in the abacus domain in a systematic way, too.
In Figure 5, we started with a crossword and created a
regular three-by-three grid of black squares by editing an
image from Figure S2. Then, systematically, we move a
white square around the grid through each of the nine po-
sitions. In each case, the TraVeLGAN generates an aba-
cus with a bead moving around the grid appropriately. Re-
markably, it even colors the bead to ﬁt with the neighboring
beads, which differ throughout the grid. Given that none

erate its paired image.
As a consequence of these differences, the TraVeLGAN is
better able to handle mappings between complex, hetero-
geneous domains that require signiﬁcant and diverse shape
changing.
By avoiding direct regularization of the generators, the
TraVe